# Regresszió

Olvasd el a regressziós gépi tanulási feladatot bevezető [előadás olvasóleckét](http://www.inf.u-szeged.hu/~rfarkas/ML20/regression.html)!


## Egy regressziós feladat ajánlórendszerekhez

Az ajánlórendszerek célja, hogy ajánlanak olyan tartalmat (film, video-on-demand, zene, könyv, hír, kép, weboldal, cikk, stb.) egy konkrét felhasználónak, amely nagy valószínűséggel érdekes lesz neki. Ehhez általában a rendszerek felhasználói- és termékprofilokat építenek a múltbeli információk alapján gépi tanuló algoritmusok segítésével. Ha többet szeretnél megtudni az ajánlórendszerekről, [lásd](http://www.inf.u-szeged.hu/~rfarkas/ML17/10a_Ajanlo_rendszerek.ppt).

A regressziós feladathoz  a [[MovieLens adatbázis]](https://grouplens.org/datasets/movielens/)t fogjuk használni, ami egy közismert adatbázis ajánlórendszerek fejlesztésére.

In [ ]:
import pandas as pd
# egy kezelhető méretű részlete a MovieLens adatbázisnak:
ratings = pd.read_csv('https://raw.githubusercontent.com/rfarkas/student_data/main/MovieLens/movielens.small.ratings') # egy kezelhető méretű részlete a MovieLens adatbázisnak
ratings.head()

A belolvasás NEM jó!

Egyrészt TAB az elválasztó karakter, márészt nincs "fejléc", az első rekorot hiszi a rendszer oszlopneveknek...

In [ ]:
# oszlopnevek:
rating_colnames = ['userID', # felhasználó azonosítója
                   'movieID', # film azonosítója
                   'unix_timestamp', # időpont amikor a felhasználó értékelte a filmet
                   'rating' # értékelés 1-5 skálán, 5 a legjobb
                   ] 

#helyes beolvasás:
ratings = pd.read_csv('https://raw.githubusercontent.com/rfarkas/student_data/main/MovieLens/movielens.small.ratings', sep='\t', names=rating_colnames) 
ratings.head()

In [ ]:
ratings.shape # 80ezer értékelés (minden rekord egy felhasználó értékelése egy konkrét filmről)

In [ ]:
# Több információt tudunk a felhasználókról?
users = pd.read_csv('https://raw.githubusercontent.com/rfarkas/student_data/main/MovieLens/movielens.user.info', sep='\t')
users.head()

In [ ]:
pd.merge(ratings, users).head() # userID-val összeköthetjük az értékeléseket és a felhasználókról szoló információkat
# ez igencsak redundáns, de ilyen kis adaton kényelmes használni...

In [ ]:
# Filmekről is rendelkezésre áll több információ:
item_colnames = ['movieID', # ezzel tudjuk összekötni az értékelésekkel
                 'title', # film címe, aztán zárójelben a bemutató éve
                 'genre', # műfaj címkék
                 'plot', # rövid tartalmi leírás
                 'actors' # színészek
                 ] 

# szöveges fájloknál a karakterkódolásra (encoding) is figyeljünk!
movies = pd.read_csv('https://raw.githubusercontent.com/rfarkas/student_data/main/MovieLens/movielens.item.info', sep='\t', names=item_colnames, encoding='latin-1')
movies.head()

In [ ]:
movies.shape

Ha a három adatbázist összefésüljük több információnk lesz a jellemzőtér kialakításához...

In [ ]:
lens = pd.merge(ratings, movies)
lens = pd.merge(lens, users)
lens.head()

Az **ajánlórendszer feladata** annak predikciója, hogy egy konkrét felhasználó, egy konkrét filmet mennyire értékelne. Azaz egy userID-movieID párosra kell egy 1..5 intervallumba eső értéket jósolnunk. Ha ezt jól meg tudjuk oldalni, az ajánlás már lehet a legnagyobb predikált ratinggel rendelkező termékek (amikre ismert a rating, az már nem érdekli a felhasználót). Az ún. tartalomalapú ajánló rendszerek a termékek tulajdonsági ("tartalma") alapján akarják megtanulni, hogy egyetlen felhasználónak mi tetszik. 

A gépi tanulás egyedei itt egy konkrét felhasználó egy konkrét termékre vonatkozó értékelései. A tanító adatbázisban kizárólag a kérdéses felhasználó korábbi értékelései vannak és a jellemzőkészlet a termékek metaadatai (pl. azt akarjuk megtanulni, hogy X felhasználó azokat a filmeket szereti/nem szereti amelyek A műfajűak és amiben XYZ színész szerepel)

In [ ]:
ratings.groupby('userID').size() # találomra válasszunk egy felhasználót akinek sok értékelése van

In [ ]:
df=lens[lens.userID==5] # 5-ös usert akarjuk modellezni
df.head()

In [ ]:
# 5-ös user értékelései lesznek most a célváltozó
df.rating.hist(bins=5)

In [ ]:
# Filmműfajok legyenek a jellemzők! Azaz azt akarjuk gépi tanulni, hogy 5-ös user milyen műfajú filmeket (nem) kedvel
# A vesszővel elválasztott string-ekből jellemzőket kell gyártanunk.
# Legegyszerűbb ha szövegként kezeljük a genre felsorolást és a CountVectorizer megoldja nekünk
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer() 
features = count.fit_transform(df.genre)

In [ ]:
features

In [ ]:
# véletlenszerű train/test vágás kiértékelésre
from sklearn.model_selection import train_test_split

trainFeatures,testFeatures,trainLabels,testLabels = train_test_split(features, df.rating, test_size=.30)
trainFeatures.shape # 338 egyed, 20 jellemző (különböző műfajok)

In [ ]:
# Regressziós döntési fát érdemes használnunk, hiszen csak 20 bináris jellemzőnk van
from sklearn.neighbors import KNeighborsRegressor

dt = KNeighborsRegressor(n_neighbors=9) # döntési fa regresszióra
dt.fit(trainFeatures, trainLabels) # tanítás a tanító adatbázison
prediction = dt.predict(testFeatures) # predikció a teszt adatbázison

In [ ]:
prediction

Regresszió esetén a kiértékelési metrika legtöbbször a [root mean square error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation ), ami az igazi és predikált értékek különbségeinek négyzetének összege.

In [ ]:
from sklearn.metrics import mean_squared_error # MSE (RMSE végső gyökvonás nélkül)
mean_squared_error(prediction, testLabels)

Mennyire jó az ekkora hiba?

Számoljuk ki az MSE-t egyszerű (baseline) jóslásra is! Ne felejtsük el, hogy csak akkor tanultunk bármit is, ha egyszerű szabályoknál jobban teljesít a rendszerünk! Az MSE egy hiba, azaz minél kisebb anál jobb.

In [ ]:
from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy='mean') # tanító adatbázis címkéinek átlaga lesz mindig a predikció
dummy.fit(trainFeatures, trainLabels)
mean_squared_error(dummy.predict(testFeatures), testLabels)

Regressziós döntési fát ugyanúgy vizualizálhatjuk, mint az osztályozásit.

In [ ]:
!apt-get -qq install -y graphviz && pip install -q pydot #Thank you https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d
!pip install graphviz

In [ ]:
import pydot
import graphviz
from sklearn import tree

graphviz.Source(tree.export_graphviz(dt, out_file=None, feature_names=count.get_feature_names()))

Figyeljük meg, hogy a predikció most egy konstans (`value`) minden levélen.
Az 5-ös user a drámákat 3.75-re, míg az akciófilmeket 3.0-ra értékeli átlagosan.

## Egy idősor előrejelzési regressziós feladat

Az **idősorok** olyan adathalmazok, ahol az időbeliség nagyon fontos, az egyes mérési pontok egy időpillanathoz köthetőek. Idősorokat leggyakrabban azért elemzünk, hogy:

*   Előrejelzzünk jövőbeli értékeket, mint például meteorológia vagy
*   hasonló idősorokat keresssünk (például kik azok a sessionök egy weboldalon, akik az aktuális sessionhöz hasonló kattintássorozaton mentek át) vagy 
*   milyen hosszútávú trendek és szezonalitás figyelhető meg az adatsorban.

Töltsünk le egy idősort a [Google Trends](https://trends.google.com/trends/explore?date=all&q=aj%C3%A1nd%C3%A9k)ből! Ez azt mutatja meg, hogy az "ajándék" szóra hogyan alakult a keresések száma ([0-100]ra normalizált érték) egy adott hónapban a Google Searchben.

(lásd ezt a [cikket](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92), hogy hogyan lehet egy lokális fájlt a colabon használni)

In [ ]:
import pandas as pd
df = pd.read_csv("multiTimeline.csv")
df.head()

In [ ]:
df.shape

Nem jó a beolvasás mert csak 1 oszlopunk lett. Érdemes a csv-t megnyitni mielőtt beolvassuk, hogy ellenőrízzük a szerkezetét...

In [ ]:
# hagyjuk ki az első 3 sort és adjunk könnyen kezelhető neveket az oszlopoknak:
df = pd.read_csv("multiTimeline.csv", skiprows=3, names=['time','data'])
df.head()

In [ ]:
df.shape

In [ ]:
# Jelenleg a time oszlopunk csak string. Konvertáljuk át igazi dátum típussá:
df.time = pd.to_datetime(df.time)
print(df.time.min() , "\n" , df.time.max())

In [ ]:
# ha az idő lesz az index ("sorok nevei") akkor könnyebben tudjuk majd használni a dataframe-et:
df = df.set_index('time')
df.head()

In [ ]:
# például a vizualizáció egyszerű:
df.plot(figsize=(15, 6)) # figsize-al jobban látható az ábra

In [ ]:
# zoomoljunk az utolsó három évre
df.loc['2017-04-20':].plot() # vegyük észre, hogy 2017-04-20 dátum nincs benne, tényleg, mint időpont kezeli az adatokat

Az idősor előrejelzés célja, hogy jövőbeli adatpontok értékét megjósoljuk a múltbeli adatok, összefüggések felhasználásával. Ez egy speciális regressziós feladat.

A problémát fel lehet fogni gépi tanulási regressziós feladatként is. Itt ha a `t` időpontban vagyunk:

*   egy egyed egy adatpont amire jósolni szeretnénk (pl. t+1 időpont)
*   jellemzőket az idősor `t` időpontot megelöző részéből nyerhetünk ki (plusz az idősoron kívüli információk)
*   regressziós problémaként fogjuk fel, a múlbeli `t` időpontok egyedein tanítunk egy gépi tanuló modellt és predikálunk a jövőre vonatkozólag


In [ ]:
# Egy előrejelzési 'egyed' vizualizációja

from dateutil.relativedelta import relativedelta

t = pd.to_datetime('2018-04-01') # tfh itt vagyunk az időben 
known = df.loc[t-relativedelta(years=3) : t-relativedelta(months=1)] # 3 éves múlt
unknown = df.loc[t-relativedelta(months=1) : t+relativedelta(years=1)] # 1 éves jövő t-től, nem ismerjük
to_predict = df.loc[t:t] # erre a pontra akarunk jósolni

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(15, 6))
known.plot(ax=ax, c='c', marker='o', zorder=3) # ismert múlt = egyedet leíró jellemzők
unknown.plot(ax=ax, c='grey', alpha=0.5) # jövő amit ismeretlennek tekintünk
to_predict.plot(ax=ax, c='r', marker='o', markersize=16, 
                linestyle='') # predikálandó/előrejelzendő folytonos célértékek

ax.legend(['múlt', 'jövő', 'predikálandó'])
plt.show()


In [ ]:
### 8db előrejelzési egyed vizualizációja 
### Az így gyártott egyedeket alkotják majd a tanító és kiértékelő adatbázist a gépi tanuláshoz

fig, ax = plt.subplots(figsize=(12, 9))

full = df.loc['2017-02-01':'2020-04-01']

for i in range(8):
    t = pd.to_datetime('2019-03-01') + relativedelta(months=i)
    predict = df.loc[t : t]
    known = df.loc[t-relativedelta(years=2) : t-relativedelta(months=1)] # 2 éves múlt

    (full + 20*i).plot(ax=ax, c='grey', alpha=0.5);
    (known + 20*i).plot(ax=ax, c='c', markersize=4,
                       marker='o')
    (predict + 20*i).plot(ax=ax, c='r', markersize=8,
                         marker='o', linestyle='')

ax.get_yaxis().set_ticks([]) # y tengely értékkészletének most nincs értelme, kikapcsoljuk
ax.legend(['teljes idősor', 'jellemzők', 'célváltozó'], bbox_to_anchor=(1, 1))


In [ ]:
### jellemzőkinyerés

target_values = [] # regresszió célváltozója (címke/label)
features = [] # egyedek jellemzői

t = df.index.min() + relativedelta(years=3)
while t < df.index.max():
  target_values.append( df.loc[t : t].data ) # célváltozó a t időpontbeli érték
  
  known = df.loc[t-relativedelta(years=3) : t-relativedelta(months=1)] # 3 éves múlt
  features.append( known.data.tolist() ) # az elmúlt 36 hónap értékei lesznek a 36 jellemző

  t = t + relativedelta(months=1)

len(features) # ennyi egyedünk van

In [ ]:
### tanító és kiértékelő adatbázisra vágjuk
### nagyon vigyázunk arra, hogy a kiértékelő adatbázisbeli egyedekről ne tudjunk meg információt a tanító adatbázisbeli elemekből
### időben vágás a "legbiztonságosabb", értékeljünk ki az időben utolsó 30 ponton!

trainFeatures = features[:-30]
testFeatures  = features[-30:]
trainLabels = target_values[:-30]
testLabels  = target_values[-30:]

In [ ]:
### regressziós gépi tanulási kísérlet
from sklearn.linear_model import LinearRegression

reg = LinearRegression() # lineáris gép regresszióra
reg.fit(trainFeatures, trainLabels)
prediction = reg.predict(testFeatures)

from sklearn.metrics import mean_squared_error # MSE
mean_squared_error(prediction, testLabels)

In [ ]:
# A baseline itt most nagyon rosszul teljesít!
dummy = DummyRegressor(strategy='mean') # tanító adatbázis címkéinek átlaga lesz mindig a predikció
dummy.fit(trainFeatures, trainLabels)
mean_squared_error(dummy.predict(testFeatures), testLabels)

In [ ]:
### vizualizáljuk a predikciót

fig, ax = plt.subplots(figsize=(18, 6))
plt.plot(range(0,len(trainLabels)), trainLabels, c='grey', alpha=0.5)
testrange = range(len(trainLabels)-1,len(trainLabels)+len(testLabels)-1)
plt.plot(testrange, testLabels, c='c', marker='o')
plt.plot(testrange, prediction, c='r', marker='o')
ax.legend(['múlt', 'jövőbeli tényleges', 'predikció'])
plt.show()

# Gyakorló feladat

Az https://archive.ics.uci.edu/ml/machine-learning-databases/00432/Data/News_Final.csv adatbázis tartalmazza (többek közt), hogy egy megjelent újságcikket, a rákövetkező héten hányan likeoltak a Facebookon. Hajts végre egy gépi tanulási kísérletet, hogy megtudjuk, hogy a `headline` szövege és a `source` alapján mennyire jól lehet megjósolni a `Facebook` likeok számát!

Csak azokat a cikkeket használd, ahol pozitív a `Facebook` likeok száma! Tanító adatbázisnak használd a 2016 máriusáig megjelent (`PublishDate`) cikkeket és értékeld ki a 2016. ápr 1. utáni cikkeken!

Vigyázz, mert 93K egyedes adatbázis! Tipp: szűrd meg a tanító adatbázist vagy az egyedek számát, vagy a jellemzőteret (szózsák modell esetén lásd a `CountVectorizer` `min_df` argumentumát) érdemes csökkenteni!